# Inizializzazione pySpark
Tramite il package `findspark` inizializzo `SparkContext`

In [ ]:
import findspark

findspark.init()

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

conf = SparkConf().setAppName("LID Project").setMaster("local")
sc = SparkContext(conf=conf)
ss = SparkSession(sc)

# Caricamento del Dataset
## Import tramite lettura del file .fvecs
Proviamo in primis a caricare il Dataset, un `np.ndarray` di dimensione 10000x128. Ogni entry è di `dtype=np.float32`

In [2]:
from utils.read import read_fvecs

dataset = read_fvecs("data/siftsmall_base.fvecs")

## Creazione matrice
Ora tramite `parallelize()` creiamo l'RDD che conterrà la matrice. Andranno fatte delle operazioni preliminari per trasformare ogni entry nel dataset in una `MatrixEntry`, dato che dovremo raggrupparle in una `CoordinateMatrix`

In [ ]:
import numpy as np
from pyspark.mllib.linalg.distributed import CoordinateMatrix, MatrixEntry

entries_list = [MatrixEntry(coordinates[0], coordinates[1], entry) for coordinates, entry in np.ndenumerate(dataset)]

entries = sc.parallelize(entries_list)

dataset_matrix = CoordinateMatrix(entries=entries)